# 1.ライブラリの読み込み
上から順に
* グラフを書くためにmatplotlib
* データの整理のためにpandas
* 数学的な計算のためにnumpy
* 時系列データを扱うためにdatetime
* matplotlibでdatetimeのフォーマットを変えるためにmdates

In [37]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.dates as mdates
from scipy import signal

# 2.センサーデータの読みこみと確認
ファイルの読み込みに使用した変数名の命名則については以下の通りである．

例　セッション1のAさんのセンサデータ →　data1_a
　　セッション1のAさんのスケーリングデータ　→　data1_a_label

In [38]:
%time
#sensor_dataの読み込み
data1_a = pd.read_csv('sensor_data/session1/TSND151/raw_data_time/心筋電/Conv-mem-A心筋-20201111-121748240-ead.csv', encoding="shift-jis")
data1_b = pd.read_csv('sensor_data/session1/TSND151/raw_data_time/心筋電/Conv-mem-B心筋-20201111-122303403-ead.csv', encoding="shift-jis")
data1_c = pd.read_csv('sensor_data/session1/TSND151/raw_data_time/心筋電/Conv-mem-C心筋-20201111-121748453-ead.csv', encoding="shift-jis")
data1_d = pd.read_csv('sensor_data/session1/TSND151/raw_data_time/心筋電/Conv-mem-D心筋-20201111-121748555-ead.csv', encoding="shift-jis")
data2_a = pd.read_csv('sensor_data/session2/TSND151/raw_data_time/心筋電/Conv-mem-A心筋-20201111-145808287-ead.csv', encoding="shift-jis")
data2_b = pd.read_csv('sensor_data/session2/TSND151/raw_data_time/心筋電/Conv-mem-B心筋-20201111-145741908-ead.csv', encoding="shift-jis")
data2_c = pd.read_csv('sensor_data/session2/TSND151/raw_data_time/心筋電/Conv-mem-C心筋-20201111-150032244-ead.csv', encoding="shift-jis")
data2_d = pd.read_csv('sensor_data/session2/TSND151/raw_data_time/心筋電/Conv-mem-D心筋-20201111-145749360-ead.csv', encoding="shift-jis")

# Anotation_data
data1_a_label = pd.read_csv('emotion_label/label/emotion_A_20201111_1.csv')
data1_b_label = pd.read_csv('emotion_label/label/emotion_B_20201111_1.csv')
data1_c_label = pd.read_csv('emotion_label/label/emotion_C_20201111_1.csv')
data1_d_label = pd.read_csv('emotion_label/label/emotion_D_20201111_1.csv')
data2_a_label = pd.read_csv('emotion_label/label/emotion_A_20201111_2.csv')
data2_b_label = pd.read_csv('emotion_label/label/emotion_B_20201111_2.csv')
data2_c_label = pd.read_csv('emotion_label/label/emotion_C_20201111_2.csv')
data2_d_label = pd.read_csv('emotion_label/label/emotion_D_20201111_2.csv')

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 2.86 µs


## 2.1 TSND151加速度，角速度センサデータとスケーリングデータの中身
今回のセンサデータの列はセンサの種類，時間，加速度のx，y，z，角速度x，y，zで構成されており，加速度の単位は0.1mG，角速度の単位は0.01dpsである．

スケーリングデータの列は，レコーディング動画の経過時間，感情のレベル値，発話されたユーザ名となっている．emotion_levelの感情値については今回は以下のように考えた．

-4〜-1→negative  0→neutral  １〜4→positive

In [39]:
#生データの確認
display(data1_a)
display(data1_c_label)

,種別,時刻,端子1AD値,端子2AD値,端子3AD値,端子4AD値,端子1AD値(フィルタあり),端子2AD値(フィルタあり),端子3AD値(フィルタなし),端子4AD値(フィルタなし)
0,ead2200,12:17:48.244,0,0,0,0,0,0,0,0
1,ead2200,12:17:48.245,0,0,0,0,0,0,0,0
2,ead2200,12:17:48.246,0,0,0,0,0,0,0,0
3,ead2200,12:17:48.247,0,0,0,0,0,0,0,0
4,ead2200,12:17:48.248,32767,28160,0,0,32111,27596,0,0
...,...,...,...,...,...,...,...,...,...,...
2438248,ead2200,12:58:26.492,-1962,-829,0,0,1234,-870,0,0
2438249,ead2200,12:58:26.493,-1966,-1035,0,0,1205,-1055,0,0
2438250,ead2200,12:58:26.494,-1929,-1358,0,0,1217,-1350,0,0
2438251,ead2200,12:58:26.495,-1886,-1473,0,0,1235,-1436,0,0


,timeHMS,emotion_level,subject
0,5:36,3,A
1,6:08,3,B
2,6:14,3,A
3,6:25,2,B
4,6:35,3,D
5,6:45,3,A
6,6:58,4,B
7,8:10,3,D
8,8:30,2,B
9,9:00,3,A


In [40]:
def drop_order(data):
    drop_data = data.drop(["端子3AD値", "端子4AD値", "端子1AD値(フィルタあり)", "端子2AD値(フィルタあり)","端子3AD値(フィルタなし)", "端子4AD値(フィルタなし)"], axis=1, inplace=False)
    return drop_data

data1_a = drop_order(data1_a)
data1_b = drop_order(data1_b)
data1_c = drop_order(data1_c)
data1_d = drop_order(data1_d)
data2_a = drop_order(data2_a)
data2_b = drop_order(data2_b)
data2_c = drop_order(data2_c)
data2_d = drop_order(data2_d)

In [41]:
data2_d

,種別,時刻,端子1AD値,端子2AD値
0,ead2200,14:57:49.363,0,0
1,ead2200,14:57:49.364,0,0
2,ead2200,14:57:49.365,0,0
3,ead2200,14:57:49.366,0,0
4,ead2200,14:57:49.367,-15360,32767
...,...,...,...,...
2584103,ead2200,15:40:53.466,5425,-12756
2584104,ead2200,15:40:53.467,4473,-11638
2584105,ead2200,15:40:53.468,3799,-10961
2584106,ead2200,15:40:53.469,3672,-10998


In [42]:
def volt_seiryu(data):
    data["電圧"] = data["端子2AD値"] * 0.073
    data["整流化"] = np.sqrt(data['電圧'] * data['電圧'])
    return data

data1_a = volt_seiryu(data1_a)
data1_b = volt_seiryu(data1_b)
data1_c = volt_seiryu(data1_c)
data1_d = volt_seiryu(data1_d)
data2_a = volt_seiryu(data2_a)
data2_b = volt_seiryu(data2_b)
data2_c = volt_seiryu(data2_c)
data2_d = volt_seiryu(data2_d)

In [43]:
data1_a

,種別,時刻,端子1AD値,端子2AD値,電圧,整流化
0,ead2200,12:17:48.244,0,0,0.000,0.000
1,ead2200,12:17:48.245,0,0,0.000,0.000
2,ead2200,12:17:48.246,0,0,0.000,0.000
3,ead2200,12:17:48.247,0,0,0.000,0.000
4,ead2200,12:17:48.248,32767,28160,2055.680,2055.680
...,...,...,...,...,...,...
2438248,ead2200,12:58:26.492,-1962,-829,-60.517,60.517
2438249,ead2200,12:58:26.493,-1966,-1035,-75.555,75.555
2438250,ead2200,12:58:26.494,-1929,-1358,-99.134,99.134
2438251,ead2200,12:58:26.495,-1886,-1473,-107.529,107.529


# 3.スケーリングデータに即したセンサデータの抽出
スケーリングデータに即したセンサデータの取得方法は以下の通りである．
1. センサデータ及びスケーリングデータの時刻列をdatetime化
2. timedelta関数を使用して，時刻計算を行い，実験実施日に合わせていく
3. センサデータの時刻列をインデックス化

  時刻列をインデックスにすることで，時間を使ってのデータのスライスが可能となる

4. スケーリングデータの時刻列を文字列型のリストにする
5. ４の操作を行うことでセンサデータからスケーリングデータに即したデータを抽出するのが容易になる

In [44]:
def datetime_order(sensor_data):
    sensor_data['時刻'] = pd.to_datetime(sensor_data['時刻'], format='%H:%M:%S.%f')
    sensor_data['時刻'] = sensor_data['時刻'] + dt.timedelta(days=44144)

datetime_order(data1_a)
datetime_order(data1_b)
datetime_order(data1_c)
datetime_order(data1_d)

datetime_order(data2_a)
datetime_order(data2_b)
datetime_order(data2_c)
datetime_order(data2_d)

In [45]:
#emotion_labelのtimeHMS列をdatetime型に変換
data1_a_label['timeHMS'] = pd.to_datetime(data1_a_label['timeHMS'], format='%M:%S')
data1_b_label['timeHMS'] = pd.to_datetime(data1_b_label['timeHMS'], format='%M:%S')
data1_c_label['timeHMS'] = pd.to_datetime(data1_c_label['timeHMS'], format='%M:%S')
data1_d_label['timeHMS'] = pd.to_datetime(data1_d_label['timeHMS'], format='%H:%M:%S')

data2_a_label['timeHMS'] = pd.to_datetime(data2_a_label['timeHMS'], format='%H:%M:%S')
data2_b_label['timeHMS'] = pd.to_datetime(data2_b_label['timeHMS'], format='%M:%S')
data2_c_label['timeHMS'] = pd.to_datetime(data2_c_label['timeHMS'], format='%H:%M:%S')
data2_d_label['timeHMS'] = pd.to_datetime(data2_d_label['timeHMS'], format='%H:%M:%S')

In [46]:
#編集したデータの確認(確認のためセッション1のAさんのデータを表示している)
display(data2_d)
display(data1_a_label)

,種別,時刻,端子1AD値,端子2AD値,電圧,整流化
0,ead2200,2020-11-11 14:57:49.363,0,0,0.000,0.000
1,ead2200,2020-11-11 14:57:49.364,0,0,0.000,0.000
2,ead2200,2020-11-11 14:57:49.365,0,0,0.000,0.000
3,ead2200,2020-11-11 14:57:49.366,0,0,0.000,0.000
4,ead2200,2020-11-11 14:57:49.367,-15360,32767,2391.991,2391.991
...,...,...,...,...,...,...
2584103,ead2200,2020-11-11 15:40:53.466,5425,-12756,-931.188,931.188
2584104,ead2200,2020-11-11 15:40:53.467,4473,-11638,-849.574,849.574
2584105,ead2200,2020-11-11 15:40:53.468,3799,-10961,-800.153,800.153
2584106,ead2200,2020-11-11 15:40:53.469,3672,-10998,-802.854,802.854


,timeHMS,emotion_level,subject
0,1900-01-01 00:05:47,1,D
1,1900-01-01 00:05:51,1,B
2,1900-01-01 00:05:58,2,C
3,1900-01-01 00:06:07,1,B
4,1900-01-01 00:06:24,0,B
5,1900-01-01 00:06:34,3,D
6,1900-01-01 00:06:57,1,B
7,1900-01-01 00:07:06,3,B
8,1900-01-01 00:07:49,2,C
9,1900-01-01 00:08:29,1,B


端子3AD値(フィルタなし)### 補足(実験実施時間)

本実験は，zoomレコーディング動画の経過時間でスケーリングデータのemotion_levelをつけたため動画の開始時刻を使って実際の実験実施時間と合わせる必要がある．

方法としては，経過時間と動画の開始時刻を足して実験実施時間を算出する．

動画の開始時間↓

第1セッション12時34分30秒開始

第2セッション15時11分44秒開始

In [47]:
#label_dataを実験日に合わせる(datetime.timedeltaを用いている)
data1_a_label['timeHMS'] = data1_a_label['timeHMS'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)
data1_b_label['timeHMS'] = data1_b_label['timeHMS'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)
data1_c_label['timeHMS'] = data1_c_label['timeHMS'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)
data1_d_label['timeHMS'] = data1_d_label['timeHMS'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)

data2_a_label['timeHMS'] = data2_a_label['timeHMS'] + dt.timedelta(days=44144, hours=15, minutes=11, seconds=44)
data2_b_label['timeHMS'] = data2_b_label['timeHMS'] + dt.timedelta(days=44144, hours=15, minutes=11, seconds=44)
data2_c_label['timeHMS'] = data2_c_label['timeHMS'] + dt.timedelta(days=44144, hours=15, minutes=11, seconds=44)
data2_d_label['timeHMS'] = data2_d_label['timeHMS'] + dt.timedelta(days=44144, hours=15, minutes=11, seconds=44)

In [48]:
#編集したデータの確認(確認のためセッション1のAさんのデータを表示している)
display(data1_a_label)

,timeHMS,emotion_level,subject
0,2020-11-11 12:40:17,1,D
1,2020-11-11 12:40:21,1,B
2,2020-11-11 12:40:28,2,C
3,2020-11-11 12:40:37,1,B
4,2020-11-11 12:40:54,0,B
5,2020-11-11 12:41:04,3,D
6,2020-11-11 12:41:27,1,B
7,2020-11-11 12:41:36,3,B
8,2020-11-11 12:42:19,2,C
9,2020-11-11 12:42:59,1,B


In [49]:
#datetime化したラベルデータを保存
data1_a_label.to_csv('emotion_label/label_datetime/emotion_A_20201111_1_datetime.csv', index=False)
data1_b_label.to_csv('emotion_label/label_datetime/emotion_B_20201111_1_datetime.csv', index=False)
data1_c_label.to_csv('emotion_label/label_datetime/emotion_C_20201111_1_datetime.csv', index=False)
data1_d_label.to_csv('emotion_label/label_datetime/emotion_D_20201111_1_datetime.csv', index=False)

data2_a_label.to_csv('emotion_label/label_datetime/emotion_A_20201111_2_datetime.csv', index=False)
data2_b_label.to_csv('emotion_label/label_datetime/emotion_B_20201111_2_datetime.csv', index=False)
data2_c_label.to_csv('emotion_label/label_datetime/emotion_C_20201111_2_datetime.csv', index=False)
data2_d_label.to_csv('emotion_label/label_datetime/emotion_D_20201111_2_datetime.csv', index=False)

In [50]:
#datetime化した加速度及び角速度データを保存
# data1_a.to_csv('sensor_data/session1/TSND151/raw_data_datetime/acc_gyr/Conv-mem-A胸-20201111-121953632-ags_datetime.csv', index=False)
# data1_b.to_csv('sensor_data/session1/TSND151/raw_data_datetime/acc_gyr/Conv-mem-B胸-20201111-122231707-ags_datetime.csv', index=False)
# data1_c.to_csv('sensor_data/session1/TSND151/raw_data_datetime/acc_gyr/Conv-mem-C胸-20201111-121953885-ags_datetime.csv', index=False)
# data1_d.to_csv('sensor_data/session1/TSND151/raw_data_datetime/acc_gyr/Conv-mem-D胸-20201111-121954011-ags_datetime.csv', index=False)

# data2_a.to_csv('sensor_data/se端子3AD値(フィルタなし)ssion2/TSND151/raw_data_datetime/acc_gyr/Conv-mem-A胸-20201111-145759193-ags_datetime.csv', index=False)
# data2_b.to_csv('sensor_data/session2/TSND151/raw_data_datetime/acc_gyr/Conv-mem-B胸-20201111-145733362-ags_datetime.csv', index=False)
# data2_c.to_csv('sensor_data/session2/TSND151/raw_data_datetime/acc_gyr/Conv-mem-C胸-20201111-150018208-ags_datetime.csv', index=False)
# data2_d.to_csv('sensor_data/session2/TSND151/raw_data_datetime/acc_gyr/Conv-mem-D胸-20201111-145806707-ags_datetime.csv', index=False)

In [51]:
#時刻をインデックスに
data1_a.set_index("時刻", inplace=True)
data1_b.set_index("時刻", inplace=True)
data1_c.set_index("時刻", inplace=True)
data1_d.set_index("時刻", inplace=True)

data2_a.set_index("時刻", inplace=True)
data2_b.set_index("時刻", inplace=True)
data2_c.set_index("時刻", inplace=True)
data2_d.set_index("時刻", inplace=True)

In [52]:
#編集したデータの確認(確認のためセッション1のAさんのデータを表示している)
display(data1_a)

,種別,端子1AD値,端子2AD値,電圧,整流化
時刻,,,,,
2020-11-11 12:17:48.244,ead2200,0,0,0.000,0.000
2020-11-11 12:17:48.245,ead2200,0,0,0.000,0.000
2020-11-11 12:17:48.246,ead2200,0,0,0.000,0.000
2020-11-11 12:17:48.247,ead2200,0,0,0.000,0.000
2020-11-11 12:17:48.248,ead2200,32767,28160,2055.680,2055.680
...,...,...,...,...,...
2020-11-11 12:58:26.492,ead2200,-1962,-829,-60.517,60.517
2020-11-11 12:58:26.493,ead2200,-1966,-1035,-75.555,75.555
2020-11-11 12:58:26.494,ead2200,-1929,-1358,-99.134,99.134


In [53]:
#label_dataのtimeHMSにおいてdatetime型をstr型にする
data1_a_label["timeHMS"] = data1_a_label["timeHMS"].dt.strftime("%Y-%m-%d %H:%M:%S")
data1_b_label["timeHMS"] = data1_b_label["timeHMS"].dt.strftime("%Y-%m-%d %H:%M:%S")
data1_c_label["timeHMS"] = data1_c_label["timeHMS"].dt.strftime("%Y-%m-%d %H:%M:%S")
data1_d_label["timeHMS"] = data1_d_label["timeHMS"].dt.strftime("%Y-%m-%d %H:%M:%S")

data2_a_label["timeHMS"] = data2_a_label["timeHMS"].dt.strftime("%Y-%m-%d %H:%M:%S")
data2_b_label["timeHMS"] = data2_b_label["timeHMS"].dt.strftime("%Y-%m-%d %H:%M:%S")
data2_c_label["timeHMS"] = data2_c_label["timeHMS"].dt.strftime("%Y-%m-%d %H:%M:%S")
data2_d_label["timeHMS"] = data2_d_label["timeHMS"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [54]:
#編集したデータの確認(確認のためセッション1のAさんのデータを表示している)
display(data1_a_label)

,timeHMS,emotion_level,subject
0,2020-11-11 12:40:17,1,D
1,2020-11-11 12:40:21,1,B
2,2020-11-11 12:40:28,2,C
3,2020-11-11 12:40:37,1,B
4,2020-11-11 12:40:54,0,B
5,2020-11-11 12:41:04,3,D
6,2020-11-11 12:41:27,1,B
7,2020-11-11 12:41:36,3,B
8,2020-11-11 12:42:19,2,C
9,2020-11-11 12:42:59,1,B


In [55]:
data1_a_time = list(data1_a_label["timeHMS"])
data1_b_time = list(data1_b_label["timeHMS"])
data1_c_time = list(data1_c_label["timeHMS"])
data1_d_time = list(data1_d_label["timeHMS"])

data2_a_time = list(data2_a_label["timeHMS"])
data2_b_time = list(data2_b_label["timeHMS"])
data2_c_time = list(data2_c_label["timeHMS"])
data2_d_time = list(data2_d_label["timeHMS"])

In [56]:
#編集したデータの確認(確認のためセッション1のAさんのデータを表示している)
data1_a_time

['2020-11-11 12:40:17',
 '2020-11-11 12:40:21',
 '2020-11-11 12:40:28',
 '2020-11-11 12:40:37',
 '2020-11-11 12:40:54',
 '2020-11-11 12:41:04',
 '2020-11-11 12:41:27',
 '2020-11-11 12:41:36',
 '2020-11-11 12:42:19',
 '2020-11-11 12:42:59',
 '2020-11-11 12:43:45',
 '2020-11-11 12:44:09',
 '2020-11-11 12:44:39',
 '2020-11-11 12:45:09',
 '2020-11-11 12:45:48',
 '2020-11-11 12:46:50',
 '2020-11-11 12:48:10',
 '2020-11-11 12:49:08',
 '2020-11-11 12:50:03',
 '2020-11-11 12:50:29',
 '2020-11-11 12:50:45',
 '2020-11-11 12:50:54',
 '2020-11-11 12:52:14',
 '2020-11-11 12:52:20',
 '2020-11-11 12:52:26',
 '2020-11-11 12:52:36',
 '2020-11-11 12:53:30',
 '2020-11-11 12:53:55',
 '2020-11-11 12:54:49']

### 抽出時間の定義

In [112]:
def time_plus_order(data_time):
    data_time_plus = pd.to_datetime(data_time)
    data_time_plus = data_time_plus + dt.timedelta(seconds=4)
    data_time_plus = pd.DataFrame(data_time_plus)
    data_time_plus = data_time_plus[0].dt.strftime("%Y-%m-%d %H:%M:%S.%f")
    data_time_plus = list(data_time_plus)
    return data_time_plus

def time_laps_order(data_time):
    data_time_laps = pd.to_datetime(data_time)
    data_time_laps = data_time_laps + dt.timedelta(seconds=9)
    data_time_laps = pd.DataFrame(data_time_laps)
    data_time_laps = data_time_laps[0].dt.strftime("%Y-%m-%d %H:%M:%S.%f")
    data_time_laps = list(data_time_laps)
    return data_time_laps

data1_a_time_plus_list, data1_a_time_laps_list = time_plus_order(data1_a_time), time_laps_order(data1_a_time)
data1_b_time_plus_list, data1_b_time_laps_list = time_plus_order(data1_b_time), time_laps_order(data1_b_time)
data1_c_time_plus_list, data1_c_time_laps_list = time_plus_order(data1_c_time), time_laps_order(data1_c_time)
data1_d_time_plus_list, data1_d_time_laps_list = time_plus_order(data1_d_time), time_laps_order(data1_d_time)

data2_a_time_plus_list, data2_a_time_laps_list = time_plus_order(data2_a_time), time_laps_order(data2_a_time)
data2_b_time_plus_list, data2_b_time_laps_list = time_plus_order(data2_b_time), time_laps_order(data2_b_time)
data2_c_time_plus_list, data2_c_time_laps_list = time_plus_order(data2_c_time), time_laps_order(data2_c_time)
data2_d_time_plus_list, data2_d_time_laps_list = time_plus_order(data2_d_time), time_laps_order(data2_d_time)

In [113]:
def sensor_segmentation_order(data, data_time_plus_list, data_time_laps_list, data_label):
    a = [data[data_time_plus_list[i]:data_time_laps_list[i]]for i in range(0, len(data_label), 1)]
    return a

data1_a_record = sensor_segmentation_order(data1_a, data1_a_time_plus_list, data1_a_time_laps_list, data1_a_label)
data1_b_record = sensor_segmentation_order(data1_b, data1_b_time_plus_list, data1_b_time_laps_list, data1_b_label)
data1_c_record = sensor_segmentation_order(data1_c, data1_c_time_plus_list, data1_c_time_laps_list, data1_c_label)
data1_d_record = sensor_segmentation_order(data1_d, data1_d_time_plus_list, data1_d_time_laps_list, data1_d_label)

data2_a_record = sensor_segmentation_order(data2_a, data2_a_time_plus_list, data2_a_time_laps_list, data2_a_label)
data2_b_record = sensor_segmentation_order(data2_b, data2_b_time_plus_list, data2_b_time_laps_list, data2_b_label)
data2_c_record = sensor_segmentation_order(data2_c, data2_c_time_plus_list, data2_c_time_laps_list, data2_c_label)
data2_d_record = sensor_segmentation_order(data2_d, data2_d_time_plus_list, data2_d_time_laps_list, data2_d_label)

# 4.特徴量算出
感情の識別を行うため学習器作成のために特徴量の算出を行う．今回特徴量としたのは，emg，VLF，LF，HFである．プログラムの処理手順を以下に示す．

1. emg
2. VLF
3. LF
4. HF1, HF2, HF3, HF4, HF5

In [114]:
data1_a_emg = []
for i in range(0, len(data1_a_label), 1):
    sum_x = np.sum(data1_a_record[i]['整流化'])
    data1_a_emg.append(sum_x)

data1_b_emg = []
for i in range(0, len(data1_b_label), 1):
    sum_x = np.sum(data1_b_record[i]['整流化'])
    data1_b_emg.append(sum_x)
    
data1_c_emg = []
for i in range(0, len(data1_c_label), 1):
    sum_x = np.sum(data1_c_record[i]['整流化'])
    data1_c_emg.append(sum_x)
    
data1_d_emg = []
for i in range(0, len(data1_d_label), 1):
    sum_x = np.sum(data1_d_record[i]['整流化'])
    data1_d_emg.append(sum_x)

data2_a_emg = []
for i in range(0, len(data2_a_label), 1):
    sum_x = np.sum(data2_a_record[i]['整流化'])
    data2_a_emg.append(sum_x)

data2_b_emg = []
for i in range(0, len(data2_b_label), 1):
    sum_x = np.sum(data2_b_record[i]['整流化'])
    data2_b_emg.append(sum_x)
    
data2_c_emg = []
for i in range(0, len(data2_c_label), 1):
    sum_x = np.sum(data2_c_record[i]['整流化'])
    data2_c_emg.append(sum_x)

data2_d_emg = []
for i in range(0, len(data2_d_label), 1):
    sum_x = np.sum(data2_d_record[i]['整流化'])
    data2_d_emg.append(sum_x)

data1_a_emg = pd.DataFrame(data1_a_emg)
data1_b_emg = pd.DataFrame(data1_b_emg)
data1_c_emg = pd.DataFrame(data1_c_emg)
data1_d_emg = pd.DataFrame(data1_d_emg)
data2_a_emg = pd.DataFrame(data2_a_emg)
data2_b_emg = pd.DataFrame(data2_b_emg)
data2_c_emg = pd.DataFrame(data2_c_emg)
data2_d_emg = pd.DataFrame(data2_d_emg)
    
emg_list = pd.concat([data1_a_emg, data1_b_emg, data1_c_emg, data1_d_emg, data2_a_emg, data2_b_emg, data2_c_emg, data2_d_emg], axis=0, ignore_index=True)
emg_list

,0
0,238927.759
1,243131.975
2,142806.834
3,226296.496
4,151643.995
...,...
264,103000.664
265,93505.335
266,237073.121
267,100522.022


# VLF

In [126]:
from scipy import fftpack
data1_a_sumVLF = []
data1_b_sumVLF = []
data1_c_sumVLF = []
data1_d_sumVLF = []
data2_a_sumVLF = []
data2_b_sumVLF = []
data2_c_sumVLF = []
data2_d_sumVLF = []

for i in range(0, len(data1_a_label), 1):
    x = data1_a_record[i]['整流化'].values.flatten()
    fs = 1000
    n = len(data1_a_record[i])
    fft = fftpack.fftfreq(n,1/fs)
    spectrum = np.abs(fftpack.fft(x))
    
    a = pd.DataFrame(fft)
    a.rename(columns={0: 'freq'},inplace = True)
    b = pd.DataFrame(spectrum)
    b.rename(columns={0: 'inten'},inplace = True)
    result = pd.concat([a,b],axis = 1)
    result = result.set_index('freq')
    VLF = result.loc[10:169]
    sumVLF = 0
    for a in VLF.inten:
        sumVLF += a
    data1_a_sumVLF.append(sumVLF)

for i in range(0, len(data1_b_label), 1):
    x = data1_b_record[i]['整流化'].values.flatten()
    fs = 1000
    n = len(data1_b_record[i])
    fft = fftpack.fftfreq(n,1/fs)
    spectrum = np.abs(fftpack.fft(x))
    
    a = pd.DataFrame(fft)
    a.rename(columns={0: 'freq'},inplace = True)
    b = pd.DataFrame(spectrum)
    b.rename(columns={0: 'inten'},inplace = True)
    result = pd.concat([a,b],axis = 1)
    result = result.set_index('freq')
    VLF = result.loc[10:169]
    sumVLF = 0
    for a in VLF.inten:
        sumVLF += a
    data1_b_sumVLF.append(sumVLF)

for i in range(0, len(data1_c_label), 1):
    x = data1_c_record[i]['整流化'].values.flatten()
    fs = 1000
    n = len(data1_c_record[i])
    fft = fftpack.fftfreq(n,1/fs)
    spectrum = np.abs(fftpack.fft(x))
    
    a = pd.DataFrame(fft)
    a.rename(columns={0: 'freq'},inplace = True)
    b = pd.DataFrame(spectrum)
    b.rename(columns={0: 'inten'},inplace = True)
    result = pd.concat([a,b],axis = 1)
    result = result.set_index('freq')
    VLF = result.loc[10:169]
    sumVLF = 0
    for a in VLF.inten:
        sumVLF += a
    data1_c_sumVLF.append(sumVLF)

for i in range(0, len(data1_d_label), 1):
    x = data1_d_record[i]['整流化'].values.flatten()
    fs = 1000
    n = len(data1_d_record[i])
    fft = fftpack.fftfreq(n,1/fs)
    spectrum = np.abs(fftpack.fft(x))
    
    a = pd.DataFrame(fft)
    a.rename(columns={0: 'freq'},inplace = True)
    b = pd.DataFrame(spectrum)
    b.rename(columns={0: 'inten'},inplace = True)
    result = pd.concat([a,b],axis = 1)
    result = result.set_index('freq')
    VLF = result.loc[10:169]
    sumVLF = 0
    for a in VLF.inten:
        sumVLF += a
    data1_d_sumVLF.append(sumVLF)

for i in range(0, len(data2_a_label), 1):
    x = data2_a_record[i]['整流化'].values.flatten()
    fs = 1000
    n = len(data2_a_record[i])
    fft = fftpack.fftfreq(n,1/fs)
    spectrum = np.abs(fftpack.fft(x))
    
    a = pd.DataFrame(fft)
    a.rename(columns={0: 'freq'},inplace = True)
    b = pd.DataFrame(spectrum)
    b.rename(columns={0: 'inten'},inplace = True)
    result = pd.concat([a,b],axis = 1)
    result = result.set_index('freq')
    VLF = result.loc[10:169]
    sumVLF = 0
    for a in VLF.inten:
        sumVLF += a
    data2_a_sumVLF.append(sumVLF)

for i in range(0, len(data2_b_label), 1):
    x = data2_b_record[i]['整流化'].values.flatten()
    fs = 1000
    n = len(data2_b_record[i])
    fft = fftpack.fftfreq(n,1/fs)
    spectrum = np.abs(fftpack.fft(x))
    
    a = pd.DataFrame(fft)
    a.rename(columns={0: 'freq'},inplace = True)
    b = pd.DataFrame(spectrum)
    b.rename(columns={0: 'inten'},inplace = True)
    result = pd.concat([a,b],axis = 1)
    result = result.set_index('freq')
    VLF = result.loc[10:169]
    sumVLF = 0
    for a in VLF.inten:
        sumVLF += a
    data2_b_sumVLF.append(sumVLF)

for i in range(0, len(data2_c_label), 1):
    x = data2_c_record[i]['整流化'].values.flatten()
    fs = 1000
    n = len(data2_c_record[i])
    fft = fftpack.fftfreq(n,1/fs)
    spectrum = np.abs(fftpack.fft(x))
    
    a = pd.DataFrame(fft)
    a.rename(columns={0: 'freq'},inplace = True)
    b = pd.DataFrame(spectrum)
    b.rename(columns={0: 'inten'},inplace = True)
    result = pd.concat([a,b],axis = 1)
    result = result.set_index('freq')
    VLF = result.loc[10:169]
    sumVLF = 0
    for a in VLF.inten:
        sumVLF += a
    data2_c_sumVLF.append(sumVLF)

for i in range(0, len(data2_d_label), 1):
    x = data2_d_record[i]['整流化'].values.flatten()
    fs = 1000
    n = len(data2_d_record[i])
    fft = fftpack.fftfreq(n,1/fs)
    spectrum = np.abs(fftpack.fft(x))
    
    a = pd.DataFrame(fft)
    a.rename(columns={0: 'freq'},inplace = True)
    b = pd.DataFrame(spectrum)
    b.rename(columns={0: 'inten'},inplace = True)
    result = pd.concat([a,b],axis = 1)
    result = result.set_index('freq')
    VLF = result.loc[10:169]
    sumVLF = 0
    for a in VLF.inten:
        sumVLF += a
    data2_d_sumVLF.append(sumVLF)    
    
    

data1_a_sumVLF = pd.DataFrame(data1_a_sumVLF)
data1_b_sumVLF = pd.DataFrame(data1_b_sumVLF)
data1_c_sumVLF = pd.DataFrame(data1_c_sumVLF)
data1_d_sumVLF = pd.DataFrame(data1_d_sumVLF)
data2_a_sumVLF = pd.DataFrame(data2_a_sumVLF)
data2_b_sumVLF = pd.DataFrame(data2_b_sumVLF)
data2_c_sumVLF = pd.DataFrame(data2_c_sumVLF)
data2_d_sumVLF = pd.DataFrame(data2_d_sumVLF)
    
sumVLF_list = pd.concat([data1_a_sumVLF, data1_b_sumVLF, data1_c_sumVLF, data1_d_sumVLF, data2_a_sumVLF, data2_b_sumVLF, data2_c_sumVLF, data2_d_sumVLF], axis=0, ignore_index=True)
sumVLF_list

,0
0,621266.113571
1,643606.671478
2,882274.654487
3,444307.091343
4,387378.068854
...,...
264,504261.866390
265,419043.651991
266,546024.058610
267,380441.791642


# LF

In [128]:
from scipy import fftpack
data1_a_sumLF = []
data1_b_sumLF = []
data1_c_sumLF = []
data1_d_sumLF = []
data2_a_sumLF = []
data2_b_sumLF = []
data2_c_sumLF = []
data2_d_sumLF = []

for i in range(0, len(data1_a_label), 1):
    x = data1_a_record[i]['整流化'].values.flatten()
    fs = 1000
    n = len(data1_a_record[i])
    fft = fftpack.fftfreq(n,1/fs)
    spectrum = np.abs(fftpack.fft(x))
    
    a = pd.DataFrame(fft)
    a.rename(columns={0: 'freq'},inplace = True)
    b = pd.DataFrame(spectrum)
    b.rename(columns={0: 'inten'},inplace = True)
    result = pd.concat([a,b],axis = 1)
    result = result.set_index('freq')
    LF = result.loc[170:200]
    sumLF = 0
    for a in LF.inten:
        sumLF += a
    data1_a_sumLF.append(sumLF)

for i in range(0, len(data1_b_label), 1):
    x = data1_b_record[i]['整流化'].values.flatten()
    fs = 1000
    n = len(data1_b_record[i])
    fft = fftpack.fftfreq(n,1/fs)
    spectrum = np.abs(fftpack.fft(x))
    
    a = pd.DataFrame(fft)
    a.rename(columns={0: 'freq'},inplace = True)
    b = pd.DataFrame(spectrum)
    b.rename(columns={0: 'inten'},inplace = True)
    result = pd.concat([a,b],axis = 1)
    result = result.set_index('freq')
    LF = result.loc[170:200]
    sumLF = 0
    for a in LF.inten:
        sumLF += a
    data1_b_sumLF.append(sumLF)

for i in range(0, len(data1_c_label), 1):
    x = data1_c_record[i]['整流化'].values.flatten()
    fs = 1000
    n = len(data1_c_record[i])
    fft = fftpack.fftfreq(n,1/fs)
    spectrum = np.abs(fftpack.fft(x))
    
    a = pd.DataFrame(fft)
    a.rename(columns={0: 'freq'},inplace = True)
    b = pd.DataFrame(spectrum)
    b.rename(columns={0: 'inten'},inplace = True)
    result = pd.concat([a,b],axis = 1)
    result = result.set_index('freq')
    LF = result.loc[170:200]
    sumLF = 0
    for a in LF.inten:
        sumLF += a
    data1_c_sumLF.append(sumLF)

for i in range(0, len(data1_d_label), 1):
    x = data1_d_record[i]['整流化'].values.flatten()
    fs = 1000
    n = len(data1_d_record[i])
    fft = fftpack.fftfreq(n,1/fs)
    spectrum = np.abs(fftpack.fft(x))
    
    a = pd.DataFrame(fft)
    a.rename(columns={0: 'freq'},inplace = True)
    b = pd.DataFrame(spectrum)
    b.rename(columns={0: 'inten'},inplace = True)
    result = pd.concat([a,b],axis = 1)
    result = result.set_index('freq')
    LF = result.loc[170:200]
    sumLF = 0
    for a in LF.inten:
        sumLF += a
    data1_d_sumLF.append(sumLF)

for i in range(0, len(data2_a_label), 1):
    x = data2_a_record[i]['整流化'].values.flatten()
    fs = 1000
    n = len(data2_a_record[i])
    fft = fftpack.fftfreq(n,1/fs)
    spectrum = np.abs(fftpack.fft(x))
    
    a = pd.DataFrame(fft)
    a.rename(columns={0: 'freq'},inplace = True)
    b = pd.DataFrame(spectrum)
    b.rename(columns={0: 'inten'},inplace = True)
    result = pd.concat([a,b],axis = 1)
    result = result.set_index('freq')
    LF = result.loc[170:200]
    sumLF = 0
    for a in LF.inten:
        sumLF += a
    data2_a_sumLF.append(sumLF)

for i in range(0, len(data2_b_label), 1):
    x = data2_b_record[i]['整流化'].values.flatten()
    fs = 1000
    n = len(data2_b_record[i])
    fft = fftpack.fftfreq(n,1/fs)
    spectrum = np.abs(fftpack.fft(x))
    
    a = pd.DataFrame(fft)
    a.rename(columns={0: 'freq'},inplace = True)
    b = pd.DataFrame(spectrum)
    b.rename(columns={0: 'inten'},inplace = True)
    result = pd.concat([a,b],axis = 1)
    result = result.set_index('freq')
    LF = result.loc[170:200]
    sumLF = 0
    for a in LF.inten:
        sumLF += a
    data2_b_sumLF.append(sumLF)

for i in range(0, len(data2_c_label), 1):
    x = data2_c_record[i]['整流化'].values.flatten()
    fs = 1000
    n = len(data2_c_record[i])
    fft = fftpack.fftfreq(n,1/fs)
    spectrum = np.abs(fftpack.fft(x))
    
    a = pd.DataFrame(fft)
    a.rename(columns={0: 'freq'},inplace = True)
    b = pd.DataFrame(spectrum)
    b.rename(columns={0: 'inten'},inplace = True)
    result = pd.concat([a,b],axis = 1)
    result = result.set_index('freq')
    LF = result.loc[170:200]
    usmLF = 0
    for a in VLF.inten:
        LF += a
    data2_c_sumLF.append(sumLF)

for i in range(0, len(data2_d_label), 1):
    x = data2_d_record[i]['整流化'].values.flatten()
    fs = 1000
    n = len(data2_d_record[i])
    fft = fftpack.fftfreq(n,1/fs)
    spectrum = np.abs(fftpack.fft(x))
    
    a = pd.DataFrame(fft)
    a.rename(columns={0: 'freq'},inplace = True)
    b = pd.DataFrame(spectrum)
    b.rename(columns={0: 'inten'},inplace = True)
    result = pd.concat([a,b],axis = 1)
    result = result.set_index('freq')
    LF = result.loc[170:200]
    sumLF = 0
    for a in LF.inten:
        sumLF += a
    data2_d_sumLF.append(sumLF)    
    
    

data1_a_sumLF = pd.DataFrame(data1_a_sumLF)
data1_b_sumLF = pd.DataFrame(data1_b_sumLF)
data1_c_sumLF = pd.DataFrame(data1_c_sumLF)
data1_d_sumLF = pd.DataFrame(data1_d_sumLF)
data2_a_sumLF = pd.DataFrame(data2_a_sumLF)
data2_b_sumLF = pd.DataFrame(data2_b_sumLF)
data2_c_sumLF = pd.DataFrame(data2_c_sumLF)
data2_d_sumLF = pd.DataFrame(data2_d_sumLF)
    
sumLF_list = pd.concat([data1_a_sumLF, data1_b_sumLF, data1_c_sumLF, data1_d_sumLF, data2_a_sumLF, data2_b_sumLF, data2_c_sumLF, data2_d_sumLF], axis=0, ignore_index=True)
sumLF_list

,0
0,40881.029796
1,41496.420792
2,82174.218359
3,32377.706810
4,26234.685656
...,...
264,30461.463670
265,29236.198526
266,30627.620440
267,24314.916564
